In [16]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Read the input.xlsx file
df = pd.read_excel('Input.xlsx')

# Loop through the URLs and extract the article text
for i, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    article = soup.find('article')
    article_text = ''
    if article:
        # Extract the article title
        title = article.find('h1').get_text()
        article_text += title + '\n\n'
        # Extract the article text
        paragraphs = article.find_all('p')
        for p in paragraphs:
            article_text += p.get_text() + '\n\n'
        # Save the article text to a file
        with open(f'{url_id}.txt', 'w', encoding='utf-8') as f:
            f.write(article_text)


In [32]:
import pandas as pd

# Read the input.xlsx file
df = pd.read_excel('output.xlsx')

# Loop through the rows and extract the required information
output = ''
for i, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    positive_score = row['POSITIVE SCORE']
    negative_score = row['NEGATIVE SCORE']
    polarity_score = row['POLARITY SCORE']
    subjectivity_score = row['SUBJECTIVITY SCORE']
    avg_sentence_length = row['AVG SENTENCE LENGTH']
    percentage_complex_words = row['PERCENTAGE OF COMPLEX WORDS']
    fog_index = row['FOG INDEX']
    avg_words_per_sentence = row['AVG NUMBER OF WORDS PER SENTENCE']
    complex_word_count = row['COMPLEX WORD COUNT']
    word_count = row['WORD COUNT']
    syllable_per_word = row['SYLLABLE PER WORD']
    personal_pronouns = row['PERSONAL PRONOUNS']
    avg_word_length = row['AVG WORD LENGTH']
    # Format the information as a string
    output += f'URL_ID: {url_id}\nURL: {url}\nPositive Score: {positive_score}\nNegative Score: {negative_score}\n'
    output += f'Polarity Score: {polarity_score}\nSubjectivity Score: {subjectivity_score}\n'
    output += f'Avg Sentence Length: {avg_sentence_length}\nPercentage of Complex Words: {percentage_complex_words}\n'
    output += f'FOG Index: {fog_index}\nAvg Number of Words per Sentence: {avg_words_per_sentence}\n'
    output += f'Complex Word Count: {complex_word_count}\nWord Count: {word_count}\n'
    output += f'Syllable per Word: {syllable_per_word}\nPersonal Pronouns: {personal_pronouns}\n'
    output += f'Avg Word Length: {avg_word_length}\n\n'

# Write the output to a text file
with open('output.txt', 'w') as f:
    f.write(output)


In [41]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [50]:
import nltk
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [51]:
import pandas as pd
import newspaper
from textblob import TextBlob

# Read the input.xlsx file
df = pd.read_excel('Input.xlsx')

# Define a function to extract article text and calculate sentiment scores
def analyze_article(url):
    try:
        # Download and parse the article
        article = newspaper.Article(url, language='en')
        article.download()
        article.parse()
        
        # Extract the article title and text
        title = article.title
        text = article.text
        
        # Calculate sentiment scores using TextBlob
        blob = TextBlob(text)
        positive_score = blob.sentiment.polarity if blob.sentiment.polarity > 0 else 0
        negative_score = -blob.sentiment.polarity if blob.sentiment.polarity < 0 else 0
        polarity_score = blob.sentiment.polarity
        subjectivity_score = blob.sentiment.subjectivity
        
        # Calculate other metrics
        sentences = blob.sentences
        avg_sentence_length = len(text) / len(sentences)
        words = blob.words
        word_count = len(words)
        complex_words = [word for word in words if len(word) > 2 and len(TextBlob(word).words) > 2]
        complex_word_count = len(complex_words)
        percentage_complex_words = 100 * complex_word_count / word_count
        fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
        avg_words_per_sentence = word_count / len(sentences)
        
        # Catch TextBlob error and return default values
        try:
            syllable_per_word = sum([len(TextBlob(word).syllables) for word in words]) / word_count
        except:
            syllable_per_word = 0
        
        personal_pronouns = sum([1 for word in words if TextBlob(word).tags[0][1] == 'PRP']) / word_count
        avg_word_length = sum([len(word) for word in words]) / word_count
        
        # Return the results as a dictionary
        return {
            'URL_ID': url.split('/')[-2],
            'URL': url,
            'POSITIVE SCORE': positive_score,
            'NEGATIVE SCORE': negative_score,
            'POLARITY SCORE': polarity_score,
            'SUBJECTIVITY SCORE': subjectivity_score,
            'AVG SENTENCE LENGTH': avg_sentence_length,
            'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
            'FOG INDEX': fog_index,
            'AVG NUMBER OF WORDS PER SENTENCE': avg_words_per_sentence,
            'COMPLEX WORD COUNT': complex_word_count,
            'WORD COUNT': word_count,
            'SYLLABLE PER WORD': syllable_per_word,
            'PERSONAL PRONOUNS': personal_pronouns,
            'AVG WORD LENGTH': avg_word_length,
        }
    except:
        # Return None if there was an error downloading or parsing the article
        return None


# Loop through the URLs and analyze each article
results = []
for url in df['URL']:
    result = analyze_article(url)
    if result is not None:
        results.append(result)

# Convert the results to a DataFrame and save to output.xlsx
output_df = pd.DataFrame(results)
output_df.to_excel('output.xlsx', index=False)


In [39]:
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=854eb8392465d5aedf7029d942a9e36f8b18078422fda995235cc0bebff80f6f
  Stored in directory: /root/.cache/pip/wheels/94/ad/df/a2a01300cea47d5695f242f7e925a805970106fd9e4b151468
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=7108ce0f8566130fa329b6bba5bd5845b1b8b3ca94e8ad2758406